## CIS 335 Assignment 2

Jensen Holm <br>
Feb. 26th 2023

In [9]:
import pandas as pd
from dataclasses import dataclass
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
)

In [12]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

train = train.drop("Unnamed: 32", axis=1)
test = test.drop("Unnamed: 32", axis=1)

train["diagnosis"] = train["diagnosis"].dropna()
test["diagnosis"]= train["diagnosis"].dropna()

X_train = train[[col for col in train.columns if col != "diagnosis"]]
Y_train = train["diagnosis"]

X_test = test[[col for col in train.columns if col != "diagnosis"]]
Y_test = test["diagnosis"]

## Model Expiramentation

In [23]:


@dataclass
class ClassifierTree:
    X_train: pd.DataFrame
    Y_train: pd.DataFrame
    X_test: pd.DataFrame
    Y_test: pd.DataFrame

    @property
    def x_train(self) -> pd.DataFrame:
        return self.X_train

    @property
    def y_train(self) -> pd.DataFrame:
        return self.Y_train
    
    @property
    def x_test(self) -> pd.DataFrame:
        return self.X_test
    
    @property
    def y_test(self) -> pd.DataFrame:
        return self.Y_test

    @staticmethod
    def norms(norm_type, depth=5):
        n = {
            "z-score": make_pipeline(
                StandardScaler(),
                DecisionTreeClassifier(max_depth=depth),
            ),
            "minmax": make_pipeline(
                MinMaxScaler(),
                DecisionTreeClassifier(max_depth=depth)
            )
        }
        try:
            return n[norm_type]
        except KeyError:
            raise KeyError(f"no such normalization type of '{norm_type}'")


    def fit(self, norm_type=None, depth=5):
        if norm_type:
            dt = self.norms(
                norm_type=norm_type, depth=depth)
        else:
            dt = DecisionTreeClassifier(max_depth=depth)

        dt.fit(self.x_train, self.y_train)
        pred = dt.predict(self.x_test)
        return accuracy_score(
            y_pred=pred,
            y_true=self.y_test,
        )



In [24]:
tree = ClassifierTree(
    X_train=X_train,
    Y_train=Y_train,
    X_test=X_test,
    Y_test=Y_test,
)

#### No Preprocessing

In [26]:
no_5 = tree.fit()
no_6 = tree.fit(depth=6)

no_6

0.4083333333333333

#### Z-Score Normalization

#### MinMax Normalization